In [1]:
from huggingface_hub import notebook_login

In [2]:
notebook_login()

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
import torch

In [4]:
torch.cuda.is_available()

True

In [5]:
raw_dataset = load_dataset("glue", "mrpc")
checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Found cached dataset glue (C:/Users/Raj/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
def tokenize_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True)

In [7]:
tokenized_datasets = raw_dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Loading cached processed dataset at C:\Users\Raj\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-8c84bcc161b9e9cc.arrow
Loading cached processed dataset at C:\Users\Raj\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-0475eb55792caa26.arrow


Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [8]:
# Training using the Trainer API
from transformers import TrainingArguments

training_args = TrainingArguments("bert-finetuned-mrpc", save_strategy="epoch", push_to_hub=True)

In [9]:
# Define the model
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [10]:
# Define the Trainer by passing in the model, the training arguments, training and validation datasets, data collator and tokenizer
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Cloning https://huggingface.co/RajkNakka/bert-finetuned-mrpc into local empty directory.


In [11]:
# Train the model using the Trainer API
trainer.train()

c:\Users\Raj\repos\hf-nlp\.venv\Lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1377 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.5748, 'learning_rate': 3.184458968772695e-05, 'epoch': 1.09}
{'loss': 0.3526, 'learning_rate': 1.3689179375453886e-05, 'epoch': 2.18}
{'train_runtime': 58.1722, 'train_samples_per_second': 189.163, 'train_steps_per_second': 23.671, 'train_loss': 0.39450008417266647, 'epoch': 3.0}


TrainOutput(global_step=1377, training_loss=0.39450008417266647, metrics={'train_runtime': 58.1722, 'train_samples_per_second': 189.163, 'train_steps_per_second': 23.671, 'train_loss': 0.39450008417266647, 'epoch': 3.0})

In [12]:
# add in compute metrics
import evaluate
import numpy as np

def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)   

In [13]:
trainer.train()

  0%|          | 0/1377 [00:00<?, ?it/s]

{'loss': 0.1243, 'learning_rate': 0.0, 'epoch': 1.09}
{'loss': 0.1157, 'learning_rate': 0.0, 'epoch': 2.18}
{'train_runtime': 67.2995, 'train_samples_per_second': 163.508, 'train_steps_per_second': 20.461, 'train_loss': 0.12236038739904599, 'epoch': 3.0}


TrainOutput(global_step=1377, training_loss=0.12236038739904599, metrics={'train_runtime': 67.2995, 'train_samples_per_second': 163.508, 'train_steps_per_second': 20.461, 'train_loss': 0.12236038739904599, 'epoch': 3.0})

In [14]:
# Evaluate the model using the Trainer API
predictions = trainer.predict(tokenized_datasets["validation"])
predictions.metrics

  0%|          | 0/51 [00:00<?, ?it/s]

{'test_loss': 0.594751238822937,
 'test_runtime': 0.5798,
 'test_samples_per_second': 703.671,
 'test_steps_per_second': 87.959}

In [15]:
# Get the predictions
preds = np.argmax(predictions.predictions, axis=-1)
preds


array([1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1,

In [16]:
# compare predictions to labels
metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8553921568627451, 'f1': 0.8984509466437176}

In [17]:
# upload the last version of the model to the hub using hf api. It will also generate a model card, relevant metadata, reporting the hyper parameters used for training, the evaluation metrics and the training script used.
trainer.push_to_hub()

To https://huggingface.co/RajkNakka/bert-finetuned-mrpc
   cc7cd99..35c9486  main -> main

